In [1]:
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()
# 2021.09.23 데이터 추출 완료 - 31140 데이터가 대부분 없음
cursor = mydb.cursor(prepared=True)
citycode = '31250'

query1 = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query1)

location_list1 = []

for a in cursor:
    location_list1.append(a)

now = datetime.now()
nowDate = now.strftime('%Y%m%d')

b_cnt = 0
for z in location_list1:
    if b_cnt == 1:
        break
        
    location = z
    query2 = ("select distinct routeid from public_busroute_info where citycode = '{}'".format(location[0]))
    cursor.execute(query2)

    location_list2 = []

    for b in cursor:
        location_list2.append(b)

    cnt = 1
    for i in location_list2:
        routeId = i

        characters = "(',')"

        join_data = ''.join( x for x in routeId if x not in characters)
        print("조회날짜 : {} | location : {} | cnt : {} | start {} public_busroute_info".format(nowDate,location[0],cnt,join_data))

        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteAcctoThrghSttnList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('pageNo') : '1',
            quote_plus('numOfRows') : '10000',
            quote_plus('routeId') : join_data
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],join_data))
            mydb.close()
            b_cnt += 1
            break
        if rows == []:
            print("location {} | {} 데이터가 없습니다.".format(location[0],join_data))
            cnt += 1
            continue
        cnt += 1
        for j in rows:
            item = j
            lat = item.findAll('gpslati')                
            lng = item.findAll('gpslong')
            nodeid = item.findAll('nodeid')
            nodenm = item.findAll('nodenm')
            nodeno = item.findAll('nodeno')
            if nodeno == []:
                nodeno = None
            else:
                nodeno = nodeno[0].text
            nodeord = item.findAll('nodeord')
            routeid = item.findAll('routeid')  
            updowncd = item.findAll('updowncd')
            
            if updowncd == []:
                updowncd = None
            elif updowncd[0].text == '0':
                updowncd = '상행'
            elif updowncd[0].text == '1':
                updowncd = '하행'

            sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd, citycode) VALUES (?,?,?,?,?,?,?,?,?)"
            val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, nodeno, nodeord[0].text, routeid[0].text, updowncd, location[0])

            cursor.execute(sql, val)
            mydb.commit()

조회날짜 : 20210923 | location : 31150 | cnt : 1 | start GGB224000051 public_busroute_info
조회날짜 : 20210923 | location : 31150 | cnt : 2 | start GGB224000053 public_busroute_info
조회날짜 : 20210923 | location : 31150 | cnt : 3 | start GGB224000055 public_busroute_info
조회날짜 : 20210923 | location : 31150 | cnt : 4 | start GGB224000056 public_busroute_info
조회날짜 : 20210923 | location : 31150 | cnt : 5 | start GGB224000057 public_busroute_info
조회날짜 : 20210923 | location : 31150 | cnt : 6 | start GGB210000027 public_busroute_info
조회날짜 : 20210923 | location : 31150 | cnt : 7 | start GGB224000002 public_busroute_info
조회날짜 : 20210923 | location : 31150 | cnt : 8 | start GGB224000004 public_busroute_info
조회날짜 : 20210923 | location : 31150 | cnt : 9 | start GGB224000005 public_busroute_info
조회날짜 : 20210923 | location : 31150 | cnt : 10 | start GGB224000011 public_busroute_info
조회날짜 : 20210923 | location : 31150 | cnt : 11 | start GGB224000027 public_busroute_info
조회날짜 : 20210923 | location : 31150 | cnt 

조회날짜 : 20210923 | location : 31190 | cnt : 24 | start GGB228000281 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 25 | start GGB228000390 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 26 | start GGB228000393 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 27 | start GGB228000396 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 28 | start GGB228000398 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 29 | start GGB228000399 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 30 | start GGB228000400 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 31 | start GGB228000401 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 32 | start GGB228000402 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 33 | start GGB228000403 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 34 | start GGB228000404 public_busroute_info
조회날짜 : 20210923 | location : 311

조회날짜 : 20210923 | location : 31190 | cnt : 117 | start GGB228000017 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 118 | start GGB234000027 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 119 | start GGB228000218 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 120 | start GGB228000273 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 121 | start GGB228000274 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 122 | start GGB228000406 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 123 | start GGB228000020 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 124 | start GGB228000079 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 125 | start GGB228000156 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 126 | start GGB228000019 public_busroute_info
조회날짜 : 20210923 | location : 31190 | cnt : 127 | start GGB228000021 public_busroute_info
조회날짜 : 20210923 | loc

조회날짜 : 20210923 | location : 31210 | cnt : 2 | start GGB230000100 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 3 | start GGB230000101 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 4 | start GGB230000106 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 5 | start GGB230000122 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 6 | start GGB230000127 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 7 | start GGB230000128 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 8 | start GGB230000131 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 9 | start GGB230000132 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 10 | start GGB230000140 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 11 | start GGB230000143 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 12 | start GGB230000144 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt

조회날짜 : 20210923 | location : 31210 | cnt : 96 | start GGB234000469 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 97 | start GGB234000476 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 98 | start GGB234000477 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 99 | start GGB234000478 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 100 | start GGB234000480 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 101 | start GGB234000523 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 102 | start GGB234000525 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 103 | start GGB234000528 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 104 | start GGB234000751 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 105 | start GGB234000753 public_busroute_info
조회날짜 : 20210923 | location : 31210 | cnt : 106 | start GGB234000756 public_busroute_info
조회날짜 : 20210923 | locatio

...

조회날짜 : 20210923 | location : 31230 | cnt : 42 | start GGB232000076 public_busroute_info
조회날짜 : 20210923 | location : 31230 | cnt : 43 | start GGB232000087 public_busroute_info
조회날짜 : 20210923 | location : 31230 | cnt : 44 | start GGB232000111 public_busroute_info
조회날짜 : 20210923 | location : 31230 | cnt : 45 | start GGB232000096 public_busroute_info
조회날짜 : 20210923 | location : 31230 | cnt : 46 | start GGB232000117 public_busroute_info
조회날짜 : 20210923 | location : 31230 | cnt : 47 | start GGB232000118 public_busroute_info
조회날짜 : 20210923 | location : 31230 | cnt : 48 | start GGB232000016 public_busroute_info
조회날짜 : 20210923 | location : 31230 | cnt : 49 | start GGB232000017 public_busroute_info
조회날짜 : 20210923 | location : 31230 | cnt : 50 | start GGB232000038 public_busroute_info
조회날짜 : 20210923 | location : 31230 | cnt : 51 | start GGB232000047 public_busroute_info
조회날짜 : 20210923 | location : 31230 | cnt : 52 | start GGB232000090 public_busroute_info
조회날짜 : 20210923 | location : 312

조회날짜 : 20210923 | location : 31240 | cnt : 77 | start GGB200000016 public_busroute_info
조회날짜 : 20210923 | location : 31240 | cnt : 78 | start GGB200000237 public_busroute_info
조회날짜 : 20210923 | location : 31240 | cnt : 79 | start GGB223000127 public_busroute_info
조회날짜 : 20210923 | location : 31240 | cnt : 80 | start GGB233000013 public_busroute_info
조회날짜 : 20210923 | location : 31240 | cnt : 81 | start GGB233000019 public_busroute_info
조회날짜 : 20210923 | location : 31240 | cnt : 82 | start GGB233000029 public_busroute_info
조회날짜 : 20210923 | location : 31240 | cnt : 83 | start GGB233000030 public_busroute_info
조회날짜 : 20210923 | location : 31240 | cnt : 84 | start GGB233000035 public_busroute_info
조회날짜 : 20210923 | location : 31240 | cnt : 85 | start GGB233000043 public_busroute_info
조회날짜 : 20210923 | location : 31240 | cnt : 86 | start GGB233000049 public_busroute_info
조회날짜 : 20210923 | location : 31240 | cnt : 87 | start GGB233000079 public_busroute_info
조회날짜 : 20210923 | location : 312

조회날짜 : 20210923 | location : 31250 | cnt : 28 | start GGB234000824 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 29 | start GGB234000849 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 30 | start GGB234000850 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 31 | start GGB234000875 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 32 | start GGB234000956 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 33 | start GGB234000960 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 34 | start GGB234000996 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 35 | start GGB234000997 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 36 | start GGB234001014 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 37 | start GGB234001017 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 38 | start GGB234001018 public_busroute_info
조회날짜 : 20210923 | location : 312

조회날짜 : 20210923 | location : 31250 | cnt : 121 | start GGB234001603 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 122 | start GGB234001609 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 123 | start GGB234001620 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 124 | start GGB234001640 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 125 | start GGB234001641 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 126 | start GGB234001644 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 127 | start GGB234001653 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 128 | start GGB234001659 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 129 | start GGB234001662 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 130 | start GGB234001664 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 131 | start GGB234001666 public_busroute_info
조회날짜 : 20210923 | loc

조회날짜 : 20210923 | location : 31250 | cnt : 215 | start GGB234001411 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 216 | start GGB234001476 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 217 | start GGB234001488 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 218 | start GGB234001489 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 219 | start GGB234001490 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 220 | start GGB234001495 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 221 | start GGB234001526 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 222 | start GGB234001559 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 223 | start GGB234001572 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 224 | start GGB234001573 public_busroute_info
조회날짜 : 20210923 | location : 31250 | cnt : 225 | start GGB234001588 public_busroute_info
조회날짜 : 20210923 | loc